<a href="https://colab.research.google.com/github/gabrielsrs/LLM-foundations-with-python/blob/master/LLM_foundations_with_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Activity Suggestion Chatbot

## Overview

This project is a chatbot-based application that suggests personalized activities based on the sentiment and emotion detected in user inputs. By analyzing a user's mood and preferences, the chatbot recommends activities to enhance productivity, relaxation, or mental well-being. If the user is unsatisfied with the suggested activity, the chatbot can generate a new one based on recent user inputs.

## Features
1. **Sentiment Analysis**:
    - Uses the **'distilbert-base-uncased-finetuned-sst-2-english'** model to detect the sentiment of the user input (positive or negative).
2. **Emotion Detection**:
    - Leverages the **'bhadresh-savani/distilbert-base-uncased-emotion'** model to identify the user's emotion (e.g., sadness, joy, anger) from the input.
3. **Activity Suggestion**:
    - Based on the detected sentiment and emotion, the app looks up an appropriate activity from a predefined dataset. The dataset maps different emotions and sentiments to various activities.
4. **Regenerate New Activity**:
    - If the user wants a different activity, the **'NousResearch/Llama-2-7b-chat-hf'** model generates a new activity based on recent inputs, including user feedback and the previously suggested activity.

## Models Used
- **Sentiment Model**:
    - **Name**: `distilbert-base-uncased-finetuned-sst-2-english`
    - **Purpose**: Analyzes user input to detect sentiment (positive/negative).
- **Emotion Model**:
    - **Name**: `bhadresh-savani/distilbert-base-uncased-emotion`
    - **Purpose**: Extracts the user's emotional state from the input to guide the chatbot in suggesting a relevant activity.
- **Activity Regeneration Model**:
    - **Name**: `NousResearch/Llama-2-7b-chat-hf`
    - **Purpose**: Generates a new activity suggestion if the user requests one, using previous inputs and context.

## Workflow

1. The user inputs a message describing their current mood or situation.
2. The system first applies sentiment analysis using **'distilbert-base-uncased-finetuned-sst-2-english'** to classify the sentiment.
3. Then, emotion detection is performed using **'bhadresh-savani/distilbert-base-uncased-emotion'** to identify the emotion behind the message.
4. Based on the detected sentiment and emotion, the chatbot looks up an appropriate activity suggestion from a dataset.
5. If the user requests another activity, the app will use **'NousResearch/Llama-2-7b-chat-hf'** to generate a fresh activity based on recent inputs and history.

## How to Use
1. **Input your current mood or situation**:
    - Example: "I've been feeling a bit restless and unproductive lately."
2. **Receive an activity suggestion**:
    - The chatbot will analyze the sentiment and emotion from your input and suggest an activity from the dataset.
3. **Regenerate an activity** (Optional):
    - If you're not satisfied with the suggestion, you can request a new activity, and the chatbot will generate one based on your recent inputs.

## Example Interaction
- **User**:
```user
Today dragged on with an overwhelming sense of boredom.
```

```bash
%% Sentiment response %%
{'label': 'Negative', 'score': 0.9990234375}

%% Emotion response %%
{'label': 'Sadness', 'score': 0.90478515625}
```
- **Chatbot**:
```chatbot
Talk to a close friend or therapist
Please let me know if you want me to generate a new activity based on your input.
```
- **User**:
```user
I Wanna another activity.
```
- **Chatbot**:
```chatbot
Sure! How about we try a new activity? Let's play a game of virtual reality escape room. It's a fun and exciting way to pass the time and challenge your problem-solving skills. Would you like to give it a try?
Please let me know if you want me to generate a new activity based on your input.
```

## Installation & Requirements

1. **Install dependencies**:
    - Install the required libraries using `pip`:
        ``
    ```bash
    !pip install accelerate protobuf sentencepiece torch git+https://github.com/huggingface/transformers huggingface_hub gradio
	```
1. **Run the app**:
    - Launch the Gradio interface to start the chatbot:
    ```googlecolab
    demo.launch()
	```

## Future Improvements

- Expand the dataset to include a wider variety of activities for different emotional states.
- Enhance the context-awareness of the chatbot for better follow-up suggestions.
- Add multilingual support to reach a broader audience.


In [31]:
!pip install accelerate protobuf sentencepiece torch git+https://github.com/huggingface/transformers huggingface_hub gradio

In [2]:
# Registred activities
activity_json = {
    "Positive": {
        "Love": {
            "Low Effort": "Watch a romantic movie",
            "Medium Effort": "Cook a special meal for a loved one",
            "High Effort": "Plan a surprise date or weekend getaway"
        },
        "Joy": {
            "Low Effort": "Listen to upbeat music",
            "Medium Effort": "Go for a walk in nature",
            "High Effort": "Organize a gathering with friends or family"
        },
        "Surprise": {
            "Low Effort": "Send a thoughtful message or gift",
            "Medium Effort": "Try a new hobby or activity",
            "High Effort": "Plan an impromptu road trip"
        },
        "Sadness": {
            "Low Effort": "Watch a feel-good movie to lift your spirits",
            "Medium Effort": "Engage in a creative hobby like painting or writing",
            "High Effort": "Volunteer to help others in need, boosting your mood"
        },
        "Anger": {
            "Low Effort": "Listen to calming music or nature sounds",
            "Medium Effort": "Do yoga or a meditation session to release tension",
            "High Effort": "Participate in a team sport or an intense workout"
        },
        "Fear": {
            "Low Effort": "Read an inspiring book or listen to a podcast on courage",
            "Medium Effort": "Take a nature walk to regain a sense of safety and calm",
            "High Effort": "Face a small fear (e.g., try something new that challenges you)"
        }
    },
    "Negative": {
        "Love": {
            "Low Effort": "Isolate and reflect on past relationships with a sad song",
            "Medium Effort": "Write a letter about your feelings, even if you don’t send it",
            "High Effort": "Confront someone about unresolved issues, knowing it could hurt"
        },
        "Joy": {
            "Low Effort": "Engage in mindless scrolling, numbing the happy feelings",
            "Medium Effort": "Overindulge in comfort food or binge-watch TV, creating guilt",
            "High Effort": "Cancel social plans last minute, causing disappointment"
        },
        "Surprise": {
            "Low Effort": "Ignore a surprise gift or compliment, dismissing the effort",
            "Medium Effort": "Respond to an unexpected event with sarcasm or pessimism",
            "High Effort": "Deliberately avoid engaging with something exciting or new"
        },
        "Sadness": {
            "Low Effort": "Watch a comforting TV show",
            "Medium Effort": "Journal about your feelings",
            "High Effort": "Talk to a close friend or therapist"
        },
        "Anger": {
            "Low Effort": "Practice deep breathing exercises",
            "Medium Effort": "Go for a run or workout",
            "High Effort": "Join a kickboxing or high-intensity fitness class"
        },
        "Fear": {
            "Low Effort": "Read a calming book or meditate",
            "Medium Effort": "Take a calming walk in a safe place",
            "High Effort": "Face a small fear (e.g., public speaking in a safe environment)"
        }
    }
}

In [3]:
# Convert dict to dataframe type
import pandas as pd
import os

data = []
for sentiment, emotions in activity_json.items():
    for emotion, efforts in emotions.items():
        for effort, activity in efforts.items():
            data.append([sentiment, emotion, effort, activity])

# Create DataFrame
activities_df = pd.DataFrame(data, columns=["Sentiment", "Emotion", "Effort", "Activity"])


In [4]:
# Define the path for the CSV file
csv_file = 'qa_activities_dataset.csv'

# Check if the CSV file exists; if not, create it with initial data
if not os.path.exists(csv_file):
    activities_df.to_csv(csv_file, index=False)
else:
    # Load the existing CSV file into a DataFrame
    activities_df = pd.read_csv(csv_file)

In [31]:
# Login Hugging Face
from huggingface_hub import login

login(token="h-----------------------------------O")

In [31]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
from torch.nn.functional import softmax

# Initialize Modelstokenizer
qa_model_name = "NousResearch/Llama-2-7b-chat-hf"
qa_model = AutoModelForCausalLM.from_pretrained(qa_model_name, torch_dtype=torch.float16, device_map="auto")
qa_tokenizer = AutoTokenizer.from_pretrained(qa_model_name)
qa_tokenizer.use_default_system_prompt = False

sentiment_model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sentiment_model = DistilBertForSequenceClassification.from_pretrained(sentiment_model_name, torch_dtype=torch.float16)
sentiment_tokenizer = DistilBertTokenizer.from_pretrained(sentiment_model_name)

emotion_model_name = "bhadresh-savani/distilbert-base-uncased-emotion"
emotion_model = DistilBertForSequenceClassification.from_pretrained(emotion_model_name, torch_dtype=torch.float16)
emotion_tokenizer = DistilBertTokenizer.from_pretrained(emotion_model_name)

# Initialize the pipeline using Hugging Face pipeline
llama_pipeline = pipeline(
    "text-generation",
    model=qa_model,
    tokenizer=qa_tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
    max_length=1024
)


In [7]:
def sentiment_analize(text):
  '''
    Analyze sentiment based in given text and return option

    Parameters:
      text (str): Text to analize sentiment

    Returns:
      sentiment (str): The sentiment by given text(Positive/Negative)
  '''
  inputs = sentiment_tokenizer(text, return_tensors="pt")
  with torch.no_grad():
    logits = sentiment_model(**inputs).logits

  predicted_class_id = logits.argmax().item()

  probabilities = softmax(logits, dim=-1)

  sentiment_analize_result = {
      "label": sentiment_model.config.id2label[predicted_class_id].capitalize(),
      "score": probabilities[0, predicted_class_id].item()
  }

  return sentiment_analize_result


In [8]:
def emotion_analize(text):
  '''
    Analyze emotion based in given text and return option

    Parameters:
      text (str): Text to analize emotion

    Returns:
      emotion (str): The emotion by given text(Love/Joy/Surprise/Sadness/Anger/Fear)
  '''
  inputs = emotion_tokenizer(text, return_tensors="pt")

  with torch.no_grad():
    logits = emotion_model(**inputs).logits

  predicted_class_id = logits.argmax().item()

  probabilities = softmax(logits, dim=-1)

  emotion_analize_result = {
      "label": emotion_model.config.id2label[predicted_class_id].capitalize(),
      "score": probabilities[0, predicted_class_id].item()
  }

  return emotion_analize_result

In [10]:
def get_activity(Sentiment, Emotion, Effort):
  """
    Retrieves an activity suggestion based on the provided sentiment, emotion, and effort level.

    Parameters:
      Sentiment (str): A sentiment gived by the model(Positive/Negative).

      Emotion (str): A specific emotion gived by the model(Love/Joy/Surprise/Sadness/Anger/Fear).

      Effort (str): The effort calculate by the score(High Effort/Medium Effort/Low Effort).

    Returns:
      (str): The activity suggestion that matches the given sentiment, emotion, and effort level from the dataset.
    """
  return activities_df[(activities_df['Sentiment'] == f'{Sentiment}') &
                  (activities_df['Emotion'] == f'{Emotion}') &
                  (activities_df['Effort'] == f'{Effort}')]['Activity'].values[0]

In [11]:
def day_analyze(resume):
  """
    Retrieves the user resume and calculate sentiment, emotion and the efort to get the correspondent activity.

    Parameters:
      resume (str): A prompt from the user.

    Returns:
      (str): The activity suggestion that matches the given sentiment, emotion, and effort level from the dataset.
    """
  sentiment_result = sentiment_analize(resume)
  emotion_result = emotion_analize(resume)

  average_score = round((sentiment_result['score'] + emotion_result['score']) / 0.2, 1)

  if average_score > 8.5:
    return get_activity(sentiment_result['label'], emotion_result['label'], 'High Effort')
  elif average_score > 6.5:
    return get_activity(sentiment_result['label'], emotion_result['label'], 'Medium Effort')
  elif average_score > 0:
    return get_activity(sentiment_result['label'], emotion_result['label'], 'Low Effort	')
  else:
    return 'None activity found'

In [28]:
def day_activity(input, history):
    """
      Generates an activity suggestion based on the user's input and conversation history.

      Parameters:
        input (str): The current input or request from the user.

        history (list): A list of lists representing the conversation history, where each list contains
          the user's previous input and the chatbot's last response.

      Returns:
        (str): A generated activity suggestion, followed by a message asking the user if they want a new activity.

    """
    if len(history) == 0:
      answer = day_analyze(input)

    else:
      last_response = history[-1][1].replace("\nPlease let me know if you want me to generate a new activity based on your input.", "")
      prompt = f"""
        request: Create a new activity and return just the new activit created, without question.
        current generated activity: {last_response}
        last input: {history[-1][0]}
        current input: {input}
      """
      activity = llama_pipeline(prompt, max_length=150, do_sample=True)[0]['generated_text']

      list_activity = activity.split("\n")
      list_prompt = prompt.split("\n")

      for item_prompt in list_prompt:
        for item_text in list_activity:
          if item_prompt == item_text:
            list_activity.remove(item_text)

      answer = '\n'.join(list_activity)

    answer = f"{answer}\nPlease let me know if you want me to generate a new activity based on your input."
    return answer


In [29]:
import gradio as gr

# Create chat instance
chat = gr.ChatInterface(
    fn=day_activity,
    title="Activity Suggestion Chatbot",
    description='<p style="text-align: center;">Tell me  about your day, and I\'ll give you a <strong>ACTIVITY</strong></p>',
)


In [31]:
# Launch app
chat.launch()